<a href="https://colab.research.google.com/github/perrin-isir/xpag-tutorials/blob/main/train_brax.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>
<a id="raw-url" href="https://raw.githubusercontent.com/perrin-isir/xpag-tutorials/main/train_brax.ipynb" download> <img align="left" src="https://img.shields.io/badge/Github-Download%20(Right%20click%20%2B%20Save%20link%20as...)-blue" alt="Download (Right click + Save link as)" title="Download Notebook"></a>

**IMPORTANT:** This notebook runs best using a GPU runtime.  
In Colab: from the Colab menu, choose Runtime > Change Runtime Type, then select **'GPU'**.

In [ ]:
import os
import sys
from IPython import get_ipython
from IPython.display import clear_output
import jax
# verifying GPU backend for jax:
assert(jax.lib.xla_bridge.get_backend().platform == 'gpu')

Flax, Brax and xpag imports:

In [ ]:
# Flax:
try:
    import flax
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install flax==0.5.0
        clear_output()
        import flax
    else:
        sys.exit("ImportError: flax not found.")
# Brax:
try:
    import brax
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install git+https://github.com/google/brax.git@v0.0.10
        clear_output()
        import brax
    else:
        sys.exit("ImportError: brax not found.")
# xpag:
try:
    import xpag
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install git+https://github.com/perrin-isir/xpag.git
        clear_output()
        import xpag
    else:
        sys.exit("ImportError: xpag not found.")

from xpag.wrappers import brax_vec_env
from xpag.buffers import JaxBuffer
from xpag.samplers import JaxSampler, HER
from xpag.setters import DefaultSetter
from xpag.agents import TD3
from xpag.tools import learn, brax_notebook_replay

We first define the training and eval environments:

In [ ]:
num_envs = 50  # the number of rollouts in parallel during training
env, eval_env, env_info = brax_vec_env('walker2d', num_envs)

We then define the agent, the sampler, the buffer and the setter (the DefaultSetter does nothing):

In [ ]:
agent = TD3(
    env_info['observation_dim'] if not env_info['is_goalenv']
    else env_info['observation_dim'] + env_info['desired_goal_dim'],
    env_info['action_dim'],
    {}
)
sampler = JaxSampler(seed=0)
buffer = JaxBuffer(
    buffer_size=200_000,
    sampler=sampler
)
setter = DefaultSetter()

We set the hyperparameters:

In [ ]:
batch_size = 256
gd_steps_per_step = 1
start_training_after_x_steps = env_info['max_episode_steps'] * 10
max_steps = 10_000_000
evaluate_every_x_steps = 5_000
save_agent_every_x_steps = 100_000
save_dir = os.path.join(os.path.expanduser('~'), 'results', 'xpag', 'train_brax')
save_episode = True
plot_projection = None

Finally, we run the training loop.  
In Colab, it should take just over 8 seconds for every 5k steps. Usually, interesting results (reward > 1000) start to occur before 100k steps,
but the variance of evaluation rollouts remains high.

In [ ]:
learn(
    env,
    eval_env,
    env_info,
    agent,
    buffer,
    setter,
    batch_size,
    gd_steps_per_step,
    start_training_after_x_steps,
    max_steps,
    evaluate_every_x_steps,
    save_agent_every_x_steps,
    save_dir,
    save_episode,
    plot_projection,
)

After stopping the training, we can replay the last evaluation episode.

In [ ]:
brax_notebook_replay(save_dir)